In [1]:
import pandas as pd
import numpy as np
data_path = '/scratch/wiensj_root/wiensj/shared_data/FIDDLE_project/'

In [2]:
examples = pd.read_csv(data_path + 'extracted/icustays.csv')
labels = pd.read_csv(data_path + 'labels/mortality.csv')

In [3]:
print("original pupulation: ", examples['ICUStayID'].nunique() )

original pupulation:  200859


In [4]:
# Remove non-adults
min_age = 18
max_age = np.inf # no max age
examples = examples[(examples.age >= min_age) & (examples.age <= max_age)]
print('Exclude non-adults', examples['ICUStayID'].nunique())
examples_ = examples

Exclude non-adults 200234


In [6]:
examples_ = examples_.rename(columns={
    'ICUStayID': 'ID'
})
examples_ = pd.merge(labels, examples_, on='ID', how='left')
examples_ = examples_[(examples_.mortality_LABEL == 0.0) | (examples_.mortality_LABEL == 1.0)]
print('Exclude unknown labels', examples_['ID'].nunique())

Exclude unknown labels 199108


In [7]:
#remove discharges and deaths before prediction time
examples_ = examples_[examples_['ICUDischargeTime'] >= 48 * 60]
print('Exclude deaths and discharges before 48h', examples_['ID'].nunique())

Exclude deaths and discharges before 48h 77066


In [8]:
examples_ = examples_[['ID', 'mortality_LABEL']]
examples_['mortality_LABEL'] = examples_['mortality_LABEL'].astype(int)

In [9]:
examples_.head()

,ID,mortality_LABEL
0,141168,1
3,141194,0
10,141233,0
11,141244,0
15,141265,0


In [10]:
examples_['mortality_LABEL'].mean()

0.1149534165520463

In [11]:
examples_.to_csv(data_path + 'population/mortality_48h.csv', index=False)